In [1]:
DATA_NAME = 'coco-outdoor' 
TRANSFORM = 'wavelet-horizontal'
CHANNEL = 'green'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_green_horizontal_wavelet_outdoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,7180.33,-0.01,-0.01,-0.02,-0.01,-0.00,-0.00,0.00
3,-0.01,1527.18,-0.00,0.01,0.00,0.00,0.00,-0.00
4,-0.01,-0.00,304.74,-0.01,-0.00,0.00,-0.00,0.00
5,-0.02,0.01,-0.01,59.79,0.00,-0.00,0.00,-0.00
6,-0.01,0.00,-0.00,0.00,11.83,-0.00,0.00,0.00
7,-0.00,0.00,0.00,-0.00,-0.00,2.38,-0.00,-0.00
8,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.53,0.00
9,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.14


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,-0.00000,-0.00001,-0.00004,-0.00004,-0.00001,-0.00004,0.00002
3,-0.00000,1.00000,-0.00001,0.00002,0.00001,0.00003,0.00000,-0.00003
4,-0.00001,-0.00001,1.00000,-0.00004,-0.00001,0.00006,-0.00003,0.00003
5,-0.00004,0.00002,-0.00004,1.00000,0.00002,-0.00000,0.00001,-0.00004
6,-0.00004,0.00001,-0.00001,0.00002,1.00000,-0.00001,0.00003,0.00006
7,-0.00001,0.00003,0.00006,-0.00000,-0.00001,1.00000,-0.00002,-0.00002
8,-0.00004,0.00000,-0.00003,0.00001,0.00003,-0.00002,1.00000,0.00002
9,0.00002,-0.00003,0.00003,-0.00004,0.00006,-0.00002,0.00002,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.04705194072355929

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[6.45112098e+06 2.85652468e+05 1.10606188e+04 4.09017030e+02
 1.50259507e+01 5.45587530e-01 2.15980626e-02 0.00000000e+00]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999476,-0.031767,-0.006025,-0.001183,-0.000235,-0.000047,-0.000011,-0.000003
1,0.031531,0.998875,-0.034781,-0.006514,-0.001286,-0.000258,-0.000058,-0.000016
2,0.007058,0.034268,0.998516,-0.040988,-0.007746,-0.001551,-0.000350,-0.000093
3,0.001659,0.007787,0.040261,0.997772,-0.051608,-0.009908,-0.002216,-0.000598
4,0.000409,0.001914,0.009613,0.050387,0.996104,-0.070029,-0.014882,-0.003982
5,0.000108,0.000504,0.002528,0.012922,0.067089,0.990391,-0.116629,-0.029150
6,0.000034,0.000157,0.000790,0.004020,0.020346,0.103694,0.958703,-0.264018
7,0.000019,0.000091,0.000453,0.002314,0.011682,0.058047,0.258960,0.964069


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0005235997614280175,
 0.0011246595917845381,
 0.001484410723548879,
 0.0022280576365736326,
 0.0038958320409903102,
 0.009609458084806621,
 0.041297136466799444,
 0.03593133317715924]